In [ ]:
# import libraries (you may add additional imports but you may not have to)
import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors
import matplotlib.pyplot as plt

In [ ]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/books/book-crossings.zip

!unzip book-crossings.zip

books_filename = 'BX-Books.csv'
ratings_filename = 'BX-Book-Ratings.csv'

In [ ]:
# import csv data into dataframes
df_books = pd.read_csv(
    books_filename,
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['isbn', 'title', 'author'],
    usecols=['isbn', 'title', 'author'],
    dtype={'isbn': 'str', 'title': 'str', 'author': 'str'})

df_ratings = pd.read_csv(
    ratings_filename,
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['user', 'isbn', 'rating'],
    usecols=['user', 'isbn', 'rating'],
    dtype={'user': 'int32', 'isbn': 'str', 'rating': 'float32'})

In [72]:
# add your code here - consider creating a new cell for each section of code
# Filter users with at least 200 ratings
user_counts = df_ratings['user'].value_counts()
df_ratings = df_ratings[df_ratings['user'].isin(user_counts[user_counts >= 200].index)]

# Filter books with at least 100 ratings
book_counts = df_ratings['isbn'].value_counts()
df_ratings = df_ratings[df_ratings['isbn'].isin(book_counts[book_counts >= 100].index)]

# Merge to get book titles
df = pd.merge(df_ratings, df_books, on='isbn')

# Create pivot table
book_user_matrix = df.pivot_table(index='title', columns='user', values='rating').fillna(0)

# Convert to sparse matrix
book_user_sparse = csr_matrix(book_user_matrix.values)

# Fit the model
model = NearestNeighbors(metric='cosine', algorithm='brute')
model.fit(book_user_sparse)

def get_recommends(book=""):
    if book not in book_user_matrix.index:
        return [book, []]

    book_index = book_user_matrix.index.get_loc(book)
    distances, indices = model.kneighbors(book_user_matrix.iloc[book_index, :].values.reshape(1, -1), n_neighbors=6)

    recommended_books = []
    for i in range(1, len(distances.flatten())):
        title = book_user_matrix.index[indices.flatten()[i]]
        distance = distances.flatten()[i]
        recommended_books.append([title, distance])

    return [book, recommended_books]


In [73]:
recommended_books = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
recommended_books

["Where the Heart Is (Oprah's Book Club (Paperback))",
 [['The Lovely Bones: A Novel', np.float32(0.7234864)],
  ["The Pilot's Wife : A Novel", np.float32(0.81926787)],
  ['The Joy Luck Club', np.float32(0.81986046)],
  ['The Notebook', np.float32(0.82366824)],
  ['Bel Canto: A Novel', np.float32(0.8247875)]]]

In [74]:
# function to return recommended books - this will be tested
def get_recommends(book = ""):


  return recommended_books

In [78]:
books = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
print(books)

def test_book_recommendation():
  test_pass = True
  recommends = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
  if recommends[0] != "Where the Heart Is (Oprah's Book Club (Paperback))":
    test_pass = False
  recommended_books = ['The Lovely Bones: A Novel',"The Pilot's Wife : A Novel",'The Joy Luck Club','The Notebook','Bel Canto: A Novel']
  recommended_books_dist = [0.72, 0.81, 0.81, 0.82, 0.82]
  for i in range(2):
    if recommends[1][i][0] not in recommended_books:
      test_pass = False
    if abs(recommends[1][i][1] - recommended_books_dist[i]) >= 0.05:
      test_pass = False
  if test_pass:
    print("You passed the challenge! 🎉🎉🎉🎉🎉")
  else:
    print("You haven't passed yet. Keep trying!")

test_book_recommendation()

["Where the Heart Is (Oprah's Book Club (Paperback))", [['The Lovely Bones: A Novel', np.float32(0.7234864)], ["The Pilot's Wife : A Novel", np.float32(0.81926787)], ['The Joy Luck Club', np.float32(0.81986046)], ['The Notebook', np.float32(0.82366824)], ['Bel Canto: A Novel', np.float32(0.8247875)]]]
You passed the challenge! 🎉🎉🎉🎉🎉
